In [1]:
# https://stackoverflow.com/questions/56115874/how-to-convert-bounding-box-x1-y1-x2-y2-to-yolo-style-x-y-w-h
def yolobbox2bbox(x,y,w,h):
    x1, y1 = round(x-w/2), round(y-h/2)
    x2, y2 = round(x+w/2), round(y+h/2)
    return x1, y1, x2, y2

In [2]:
def draw_bbox(img, cl, x1, y1, x2, y2):
    import cv2
    # color system BGR
    img = img
    if cl == 0:
        color = (0,255,0)
    elif cl == 1:
        color = (0,255,255)
    elif cl == 2:
        color = (0,0,255)
    else:
        color = (255,255,0)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, 1)
    return img

In [3]:
def draw_bbox2vdo(source, destination='slideshow', img_dir='images', label_dir='labels', filename='bbox', fps=60):
    import os, cv2
    import matplotlib.pyplot as plt
    from tqdm.notebook import tqdm
    font = cv2.FONT_HERSHEY_SIMPLEX
    src = source
    dst = os.path.join(src, destination)
    img_dir = img_dir
    label_dir = label_dir
    img_path = os.path.join(src,img_dir)
    img_files = os.listdir(img_path)
    label_path = os.path.join(src, label_dir)
    if not os.path.exists(dst):
        os.makedirs(dst)
    size = (704, 576)
    out = cv2.VideoWriter(f'{dst}/{filename}.mp4',cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
    
    for file in tqdm(img_files):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            img = cv2.imread(os.path.join(img_path,file))
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            height, width, color = img.shape
            ann_path = os.path.join(label_path,f'{file[:-3]}txt')

            if os.path.getsize(ann_path) > 0:
                with open(ann_path, 'r') as txt:
                    txt = txt.read().splitlines()

                for line in txt:
                    cl, x, y, w, h = [float(i) for i in line.split(' ')]
                    cl = int(cl)
                    x = round(x*width)
                    y = round(y*height)
                    w = round(w*width)
                    h = round(h*height)
                    x1, y1, x2, y2 = yolobbox2bbox(x,y,w,h)
                    img_with_bbox = draw_bbox(img, cl, x1, y1, x2, y2)
            else:
                img_with_bbox = img.copy()
            cv2.rectangle(img_with_bbox, (15, height-40), (400, height-80), (0, 0, 0), -1)
            cv2.putText(img_with_bbox, file[:-4], (20, height-50),
                        font, 1, (255, 255, 255), 2, cv2.LINE_AA)
            img_with_bbox = cv2.resize(img_with_bbox, size)
            out.write(img_with_bbox)
    out.release()

In [7]:
for t in ['train','valid','test']:
    src = f'V:/data_annotation_v1/original/seaview_natthapol/{t}'
    draw_bbox2vdo(source=src,label_dir='images')

  0%|          | 0/10638 [00:00<?, ?it/s]

  0%|          | 0/3038 [00:00<?, ?it/s]

  0%|          | 0/1520 [00:00<?, ?it/s]